In [ ]:
# This uses hugging face modesl
!pip install smolagents arxiv pypdf networkx matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 980.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 32.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 25.9 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=95a127f86fe5044a8ec911ffccebec8a369d2a300c1328ee738109cb3005b87f
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not 

In [ ]:
from smolagents import tool, CodeAgent, HfApiModel
import arxiv
from pypdf import PdfReader
import matplotlib.pyplot as plt
import networkx as nx
from typing import List, Dict, Any
from collections import Counter
import re
from datetime import datetime

@tool
def search_papers(query: str, max_results: int = 10) -> List[Dict[str, Any]]:
    """
    Searches for academic papers on ArXiv

    Args:
        query: Search query string representing the topic to search for
        max_results: Maximum number of papers to retrieve from ArXiv

    Returns:
        List of paper dictionaries containing title, authors, year, and abstract
    """
    papers = []
    client = arxiv.Client()
    search = arxiv.Search(
        query=query,
        max_results=max_results
    )

    for result in client.results(search):
        papers.append({
            'title': result.title,
            'authors': [str(author) for author in result.authors],
            'year': result.published.year,
            'abstract': result.summary,
            'id': result.entry_id,
            'pdf_url': result.pdf_url,
            'categories': result.categories
        })

    return papers

@tool
def analyze_business_impact(papers: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Analyzes the business implications from research papers

    Args:
        papers: List of paper dictionaries containing research information

    Returns:
        Dictionary containing business impact analysis
    """
    # Keywords for different business aspects
    application_keywords = ['application', 'industry', 'commercial', 'product', 'market', 'business']
    challenge_keywords = ['challenge', 'limitation', 'problem', 'issue', 'barrier']
    opportunity_keywords = ['opportunity', 'potential', 'promising', 'future', 'improvement']

    analysis = {
        'potential_applications': [],
        'technical_challenges': [],
        'business_opportunities': [],
        'market_trends': [],
        'adoption_timeline': {
            'short_term': [],
            'medium_term': [],
            'long_term': []
        }
    }

    # Analyze each paper
    for paper in papers:
        text = paper['title'] + ' ' + paper['abstract']
        text_lower = text.lower()

        # Extract potential applications
        for keyword in application_keywords:
            pattern = f"{keyword}.*?\\."
            matches = re.findall(pattern, text_lower)
            if matches:
                analysis['potential_applications'].extend(matches)

        # Identify technical challenges
        for keyword in challenge_keywords:
            pattern = f"{keyword}.*?\\."
            matches = re.findall(pattern, text_lower)
            if matches:
                analysis['technical_challenges'].extend(matches)

        # Find business opportunities
        for keyword in opportunity_keywords:
            pattern = f"{keyword}.*?\\."
            matches = re.findall(pattern, text_lower)
            if matches:
                analysis['business_opportunities'].extend(matches)

        # Assess implementation timeline based on paper content
        if any(word in text_lower for word in ['implemented', 'deployed', 'available', 'production']):
            analysis['adoption_timeline']['short_term'].append(paper['title'])
        elif any(word in text_lower for word in ['prototype', 'experimental', 'proposed']):
            analysis['adoption_timeline']['medium_term'].append(paper['title'])
        else:
            analysis['adoption_timeline']['long_term'].append(paper['title'])

    return analysis

@tool
def create_market_analysis(papers: List[Dict[str, Any]], business_impact: Dict[str, Any]) -> str:
    """
    Generates a market analysis report based on research papers

    Args:
        papers: List of paper dictionaries
        business_impact: Dictionary containing business impact analysis

    Returns:
        Formatted market analysis report
    """
    report = "# Market Analysis Report\n\n"

    # Overview
    report += "## Market Overview\n"
    report += f"Analysis based on {len(papers)} research papers "
    report += f"published between {min(p['year'] for p in papers)} and {max(p['year'] for p in papers)}.\n\n"

    # Technology Trends
    report += "## Technology Trends\n"
    for paper in sorted(papers, key=lambda x: x['year'], reverse=True):
        report += f"### {paper['year']}: {paper['title']}\n"
        report += f"Key Points:\n"
        report += f"- Authors: {', '.join(paper['authors'])}\n"
        # Extract key sentences from abstract containing trend-related keywords
        trend_keywords = ['trend', 'advancement', 'improvement', 'development', 'progress']
        for sentence in paper['abstract'].split('.'):
            if any(keyword in sentence.lower() for keyword in trend_keywords):
                report += f"- {sentence.strip()}.\n"
        report += "\n"

    # Market Applications
    report += "## Potential Market Applications\n"
    if business_impact['potential_applications']:
        for app in set(business_impact['potential_applications']):
            report += f"- {app.capitalize()}\n"

    # Implementation Timeline
    report += "\n## Technology Adoption Timeline\n"
    report += "\n### Short-term Opportunities (0-12 months)\n"
    for tech in business_impact['adoption_timeline']['short_term']:
        report += f"- {tech}\n"

    report += "\n### Medium-term Developments (1-3 years)\n"
    for tech in business_impact['adoption_timeline']['medium_term']:
        report += f"- {tech}\n"

    report += "\n### Long-term Possibilities (3+ years)\n"
    for tech in business_impact['adoption_timeline']['long_term']:
        report += f"- {tech}\n"

    # Challenges and Risks
    report += "\n## Technical Challenges and Business Risks\n"
    if business_impact['technical_challenges']:
        for challenge in set(business_impact['technical_challenges']):
            report += f"- {challenge.capitalize()}\n"

    # Business Opportunities
    report += "\n## Business Opportunities\n"
    if business_impact['business_opportunities']:
        for opportunity in set(business_impact['business_opportunities']):
            report += f"- {opportunity.capitalize()}\n"

    return report

@tool
def generate_executive_summary(market_analysis: str) -> str:
    """
    Creates an executive summary from the market analysis

    Args:
        market_analysis: Full market analysis report

    Returns:
        Executive summary in a business-friendly format
    """
    summary = "# Executive Summary\n\n"

    # Extract key sections from market analysis
    sections = market_analysis.split('\n## ')

    # Overview
    if len(sections) > 0:
        overview = sections[0].split('\n')[2:4]  # Skip the title
        summary += "## Overview\n"
        summary += '\n'.join(overview) + "\n\n"

    # Key Findings
    summary += "## Key Findings\n\n"

    # Extract technology trends
    summary += "### Technology Trends\n"
    trends_section = next((s for s in sections if s.startswith('Technology Trends')), '')
    if trends_section:
        trends = re.findall(r'- .*?(?=\n|$)', trends_section)[:3]  # Get top 3 trends
        for trend in trends:
            summary += f"{trend}\n"

    # Market Opportunities
    summary += "\n### Market Opportunities\n"
    opportunities_section = next((s for s in sections if s.startswith('Business Opportunities')), '')
    if opportunities_section:
        opportunities = re.findall(r'- .*?(?=\n|$)', opportunities_section)[:3]  # Top 3 opportunities
        for opp in opportunities:
            summary += f"{opp}\n"

    # Key Risks
    summary += "\n### Key Risks and Challenges\n"
    risks_section = next((s for s in sections if s.startswith('Technical Challenges')), '')
    if risks_section:
        risks = re.findall(r'- .*?(?=\n|$)', risks_section)[:3]  # Top 3 risks
        for risk in risks:
            summary += f"{risk}\n"

    return summary

def run_business_analysis(query: str, max_papers: int = 10, hf_token: str = None):
    """
    Main function to run the research-to-business analysis

    Args:
        query: Search query string
        max_papers: Maximum number of papers to analyze
        hf_token: Hugging Face API token
    """
    if not hf_token:
        raise ValueError("Please provide a Hugging Face API token")

    model = HfApiModel(
        model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
        token="your_hugging_face_token"
    )

    agent = CodeAgent(
        tools=[search_papers, analyze_business_impact,
               create_market_analysis, generate_executive_summary],
        model=model,
        add_base_tools=True
    )

    result = agent.run(
        f"""Analyze the business potential of '{query}' by:
        1. Searching for up to {max_papers} relevant research papers
        2. Analyzing business impact and market potential
        3. Creating a detailed market analysis
        4. Generating an executive summary
        Please provide updates at each step."""
    )

    return result

# Example usage
if __name__ == "__main__":
    # For testing without the agent
    try:
        # Test paper search
        papers = search_papers("autonomous vehicles business applications", max_results=3)
        print("Found papers:", len(papers))

        # Test business impact analysis
        impact = analyze_business_impact(papers)
        print("\nBusiness Impact Analysis:", impact.keys())

        # Test market analysis generation
        market_report = create_market_analysis(papers, impact)
        print("\nMarket Analysis Report Length:", len(market_report))

        # Test executive summary
        summary = generate_executive_summary(market_report)
        print("\nExecutive Summary Length:", len(summary))

    except Exception as e:
        print(f"Error during testing: {str(e)}")

Found papers: 3

Business Impact Analysis: dict_keys(['potential_applications', 'technical_challenges', 'business_opportunities', 'market_trends', 'adoption_timeline'])

Market Analysis Report Length: 1259

Executive Summary Length: 326


In [ ]:
# Set your Hugging Face token
hf_token = "your_huggingface_token"  # Replace with your actual token

# Run the full analysis
analysis = run_business_analysis(
    query="autonomous vehicles business applications",
    max_papers=5,
    hf_token=hf_token
)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Analyze the business potential of 'autonomous vehicles business applications' by:                               │
│         1. Searching for up to 5 relevant research papers                                                       │
│         2. Analyzing business impact and market potential                                                       │
│         3. Creating a detailed market analysis                                                                  │
│         4. Generating an executive summary                                                                      │
│         Please provide updates at each step.                                                                    │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  papers = search_papers(query="autonomous vehicles business applications", max_results=5)                         
  print(papers)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[{'title': 'Smart Roads: Roadside Perception, Vehicle-Road Cooperation and Business Model', 'authors': ['Rui Chen',
'Lu Gao', 'Yutian Liu', 'Yong Liang Guan', 'Yan Zhang'], 'year': 2023, 'abstract': 'Smart roads have become an 
essential component of intelligent transportation\nsystems (ITS). The roadside perception technology, a critical 
aspect of smart\nroads, utilizes various sensors, roadside units (RSUs), and edge computing\ndevices to gather 
real-time traffic data for vehicle-road cooperation. However,\nthe full potential of smart roads in improving the 
safety and efficiency of\nautonomous vehicles only can be realized through the mass deployment of\nroadside 
perception and communication devices. On the one hand, roadside\ndevices require significant investment but can 
only achieve monitoring function\ncurrently, resulting in no profitability for investors. On the other 
hand,\ndrivers lack trust in the safety of autonomous driving technology, making it\ndifficult to promote 
large-scale commercial applications. To deal with the\ndilemma of mass deployment, we propose a novel smart-road 
vehicle-guiding\narchitecture for vehicle-road cooperative autonomous driving, based on which we\nthen propose the 
corresponding business model and analyze its benefits from\nboth operator and driver perspectives. The numerical 
simulations validate that\nour proposed smart road solution can enhance driving safety and traffic\nefficiency. 
Moreover, we utilize the cost-benefit analysis (CBA) model to\nassess the economic advantages of the proposed 
business model which indicates\nthat the smart highway that can provide vehicle-guided-driving services 
for\nautonomous vehicles yields more profit than the regular highway.', 'id': 'http://arxiv.org/abs/2312.09439v1', 
'pdf_url': 'http://arxiv.org/pdf/2312.09439v1', 'categories': ['eess.SP', 'cs.RO', 'cs.SY', 'eess.SY']}, {'title': 
'Optimization of a Line Detection Algorithm for Autonomous Vehicles on a RISC-V with Accelerator', 'authors': 
['María José Belda', 'Katzalin Olcoz', 'Fernando Castro', 'Francisco Tirado'], 'year': 2024, 'abstract': 'In recent
years, autonomous vehicles have attracted the attention of many\nresearch groups, both in academia and business, 
including researchers from\nleading companies such as Google, Uber and Tesla. This type of vehicles are\nequipped 
with systems that are subject to very strict requirements, essentially\naimed at performing safe operations -- both
for potential passengers and\npedestrians -- as well as carrying out the processing needed for decision\nmaking in 
real time. In many instances, general-purpose processors alone cannot\nensure that these safety, reliability and 
real-time requirements are met, so it\nis common to implement heterogeneous systems by including accelerators. 
This\npaper explores the acceleration of a line detection application in the\nautonomous car environment using a 
heterogeneous system consisting of a\ngeneral-purpose RISC-V core and a domain-specific accelerator. In 
particular,\nthe application is analyzed to identify the most computationally intensive\nparts of the code and it 
is adapted accordingly for more efficient processing.\nFurthermore, the code is executed on the aforementioned 
hardware platform to\nverify that the execution effectively meets the existing requirements in\nautonomous 
vehicles, experiencing a 3.7x speedup with respect to running\nwithout accelerator.', 'id': 
'http://arxiv.org/abs/2402.00496v1', 'pdf_url': 'http://arxiv.org/pdf/2402.00496v1', 'categories': ['cs.AR']}, 
{'title': 'Road Quality Analysis Based on Cognitive Internet of Vehicles (CIoV)', 'authors': ['Hamed Rahimi', 
'Dhayananth Dharmalingam'], 'year': 2020, 'abstract': 'This research proposal aims to use cognitive methods to 
analyze the quality\nof roads based on the new proposed technology called Cognitive Internet of\nVehicles (CIoV). 
By using Big Data corresponding to the collected data of\nautonomo

[Step 0: Duration 2.78 seconds| Input tokens: 2,492 | Output tokens: 72]

...

[Step 2: Duration 2.17 seconds| Input tokens: 11,735 | Output tokens: 209]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  executive_summary = generate_executive_summary(market_analysis=market_analysis)                                  
  final_answer(executive_summary)                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: # Executive Summary

## Overview


## Key Findings

### Technology Trends
- Authors: María José Belda, Katzalin Olcoz, Fernando Castro, Francisco Tirado
- Authors: Lanhang Ye, Toshiyuki Yamamoto
- This work provides a

### Market Opportunities
- Future of modular mobility.
- Future mobility ecosystems.

### Key Risks and Challenges
- Problem.

[Step 3: Duration 3.48 seconds| Input tokens: 17,206 | Output tokens: 328]

In [ ]:
from smolagents import tool, CodeAgent, HfApiModel
import arxiv
from typing import List, Dict, Any
from collections import Counter
import re
from datetime import datetime

@tool
def search_papers(query: str, max_results: int = 10) -> List[Dict[str, Any]]:
    """
    Searches for academic papers on ArXiv with business relevance

    Args:
        query: Search query string
        max_results: Maximum number of papers to retrieve

    Returns:
        List of relevant papers with metadata
    """
    papers = []
    client = arxiv.Client()
    # Enhance query with business-related terms
    business_query = f"{query} AND (industry OR commercial OR business OR market OR application)"
    search = arxiv.Search(
        query=business_query,
        max_results=max_results
    )

    for result in client.results(search):
        papers.append({
            'title': result.title,
            'authors': [str(author) for author in result.authors],
            'year': result.published.year,
            'abstract': result.summary,
            'id': result.entry_id,
            'categories': result.categories
        })

    return papers

@tool
def analyze_market_potential(papers: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Analyzes market potential and business implications

    Args:
        papers: List of research papers

    Returns:
        Comprehensive market analysis
    """
    analysis = {
        'market_segments': [],
        'business_models': [],
        'revenue_streams': [],
        'key_players': [],
        'competitive_advantages': [],
        'market_drivers': [],
        'market_barriers': [],
        'maturity_assessment': {
            'research_stage': [],
            'prototype_stage': [],
            'market_ready': []
        }
    }

    # Market analysis keywords
    market_keywords = {
        'segments': ['segment', 'sector', 'industry', 'vertical', 'market'],
        'business_models': ['business model', 'monetization', 'revenue', 'commercial'],
        'advantages': ['advantage', 'unique', 'superior', 'better', 'improve'],
        'drivers': ['driver', 'trend', 'growth', 'demand', 'need'],
        'barriers': ['barrier', 'challenge', 'limitation', 'constraint', 'problem']
    }

    for paper in papers:
        text = f"{paper['title']} {paper['abstract']}"

        # Extract market segments
        for keyword in market_keywords['segments']:
            matches = re.findall(f"{keyword}.*?(?=[.!?])", text.lower())
            analysis['market_segments'].extend(matches)

        # Identify business models
        for keyword in market_keywords['business_models']:
            matches = re.findall(f"{keyword}.*?(?=[.!?])", text.lower())
            analysis['business_models'].extend(matches)

        # Extract competitive advantages
        for keyword in market_keywords['advantages']:
            matches = re.findall(f"{keyword}.*?(?=[.!?])", text.lower())
            analysis['competitive_advantages'].extend(matches)

        # Identify market drivers
        for keyword in market_keywords['drivers']:
            matches = re.findall(f"{keyword}.*?(?=[.!?])", text.lower())
            analysis['market_drivers'].extend(matches)

        # Extract market barriers
        for keyword in market_keywords['barriers']:
            matches = re.findall(f"{keyword}.*?(?=[.!?])", text.lower())
            analysis['market_barriers'].extend(matches)

        # Assess technology maturity
        if any(word in text.lower() for word in ['implemented', 'deployed', 'commercial', 'product']):
            analysis['maturity_assessment']['market_ready'].append(paper['title'])
        elif any(word in text.lower() for word in ['prototype', 'demonstration', 'testing']):
            analysis['maturity_assessment']['prototype_stage'].append(paper['title'])
        else:
            analysis['maturity_assessment']['research_stage'].append(paper['title'])

    return analysis

@tool
def generate_business_report(papers: List[Dict[str, Any]], market_analysis: Dict[str, Any]) -> str:
    """
    Generates comprehensive business report

    Args:
        papers: List of research papers
        market_analysis: Market potential analysis

    Returns:
        Detailed business report in markdown format
    """
    report = "# Business Opportunity Analysis Report\n\n"

    # Executive Summary
    report += "## Executive Summary\n"
    report += f"Analysis based on {len(papers)} research publications from {min(p['year'] for p in papers)} "
    report += f"to {max(p['year'] for p in papers)}, focusing on market potential and business opportunities.\n\n"

    # Market Overview
    report += "## Market Overview\n"
    # Group papers by year
    papers_by_year = {}
    for paper in papers:
        year = paper['year']
        if year not in papers_by_year:
            papers_by_year[year] = []
        papers_by_year[year].append(paper)

    # Recent Developments
    report += "### Recent Developments\n"
    for year in sorted(papers_by_year.keys(), reverse=True):
        for paper in papers_by_year[year]:
            report += f"- ({year}) {paper['title']}\n"
            # Extract key findings from abstract
            key_findings = re.findall(r'(?:result|show|demonstrate|achieve|improve).*?(?=[.!?])', paper['abstract'].lower())
            if key_findings:
                report += "  - Key Finding: " + key_findings[0].capitalize() + "\n"
    report += "\n"

    # Market Potential
    report += "## Market Potential\n"

    # Market Segments
    if market_analysis['market_segments']:
        report += "### Target Market Segments\n"
        unique_segments = list(set(market_analysis['market_segments']))
        for segment in unique_segments[:5]:  # Top 5 segments
            report += f"- {segment.capitalize()}\n"
    report += "\n"

    # Business Models
    if market_analysis['business_models']:
        report += "### Potential Business Models\n"
        unique_models = list(set(market_analysis['business_models']))
        for model in unique_models[:5]:  # Top 5 models
            report += f"- {model.capitalize()}\n"
    report += "\n"

    # Market Drivers & Growth Factors
    if market_analysis['market_drivers']:
        report += "### Market Drivers\n"
        unique_drivers = list(set(market_analysis['market_drivers']))
        for driver in unique_drivers[:5]:  # Top 5 drivers
            report += f"- {driver.capitalize()}\n"
    report += "\n"

    # Technology Readiness
    report += "## Technology Readiness Assessment\n"

    if market_analysis['maturity_assessment']['market_ready']:
        report += "\n### Market-Ready Solutions\n"
        for tech in market_analysis['maturity_assessment']['market_ready']:
            report += f"- {tech}\n"

    if market_analysis['maturity_assessment']['prototype_stage']:
        report += "\n### In Development (Prototype Stage)\n"
        for tech in market_analysis['maturity_assessment']['prototype_stage']:
            report += f"- {tech}\n"

    if market_analysis['maturity_assessment']['research_stage']:
        report += "\n### Early Research Stage\n"
        for tech in market_analysis['maturity_assessment']['research_stage']:
            report += f"- {tech}\n"

    # Risks and Challenges
    if market_analysis['market_barriers']:
        report += "\n## Risks and Challenges\n"
        unique_barriers = list(set(market_analysis['market_barriers']))
        for barrier in unique_barriers[:5]:  # Top 5 barriers
            report += f"- {barrier.capitalize()}\n"

    # Recommendations
    report += "\n## Recommendations\n"
    report += "Based on the analysis:\n"

    # Generate recommendations based on maturity assessment
    if len(market_analysis['maturity_assessment']['market_ready']) > len(market_analysis['maturity_assessment']['research_stage']):
        report += "- Market is mature for commercial entry\n"
        report += "- Focus on differentiation and competitive advantages\n"
    else:
        report += "- Technology still in early stages\n"
        report += "- Consider R&D investments and strategic partnerships\n"

    return report

def run_business_analysis(query: str, max_papers: int = 10, hf_token: str = None):
    """
    Main function to run the business analysis

    Args:
        query: Search query string
        max_papers: Maximum number of papers to analyze
        hf_token: Hugging Face API token
    """
    if not hf_token:
        raise ValueError("Please provide a Hugging Face API token")

    model = HfApiModel(
        model_id="Qwen/Qwen2.5-Coder-32B-Instruct",
        token="your_huggingface_token"
    )

    agent = CodeAgent(
        tools=[search_papers, analyze_market_potential, generate_business_report],
        model=model,
        add_base_tools=True
    )

    result = agent.run(
        f"""Analyze the business potential of '{query}' by:
        1. Finding {max_papers} relevant research papers
        2. Analyzing market potential and business implications
        3. Generating a comprehensive business report
        Include specific market segments, business models, and recommendations."""
    )

    return result

# Example usage
if __name__ == "__main__":
    # Test individual components
    papers = search_papers("autonomous vehicles commercial applications", max_results=3)
    market_analysis = analyze_market_potential(papers)
    report = generate_business_report(papers, market_analysis)
    print("\nBusiness Report Preview:")
    print(report[:500] + "...")  # Print first 500 characters


Business Report Preview:
# Business Opportunity Analysis Report

## Executive Summary
Analysis based on 3 research publications from 2020 to 2023, focusing on market potential and business opportunities.

## Market Overview
### Recent Developments
- (2023) Smart Roads: Roadside Perception, Vehicle-Road Cooperation and Business Model
  - Key Finding: Resulting in no profitability for investors
- (2021) From market-ready ROVs to low-cost AUVs
- (2020) Road Quality Analysis Based on Cognitive Internet of Vehicles (CIoV)
  ...


In [ ]:
analysis = run_business_analysis(
    query="autonomous vehicles",
    max_papers=3,
    hf_token="your_huggingface_token"
)
print(analysis)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Analyze the business potential of 'autonomous vehicles' by:                                                     │
│         1. Finding 3 relevant research papers                                                                   │
│         2. Analyzing market potential and business implications                                                 │
│         3. Generating a comprehensive business report                                                           │
│         Include specific market segments, business models, and recommendations.                                 │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 0 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  papers = search_papers(query="autonomous vehicles", max_results=3)                                               
  print(papers)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[{'title': 'A Survey and Insights on Deployments of the Connected and Autonomous Vehicles in US', 'authors': 
['Sanchu Han'], 'year': 2020, 'abstract': 'CV/ITS (Connected Vehicle, Intelligent Transportation System) and 
AV/ADS\n(Autonomous Vehicle, Automated Driving System) have been emerging for the sake\nof saving people lives, 
improving traffic efficiency and helping the\nenvironment for decades. There are separate efforts led respectively 
by USDOT\nwith state DOTs for CV, and private sectors through market driven approach from\nstart-ups and technology
companies for AV. By CV/ITS effort there are 97\ndeployments of V2X communications utilizing the 5.9 GHz band, 
18,877 vehicles\nwith aftermarket V2X communications devices, and 8,098 infrastructure V2X\ndevices installed at 
the roadsides. However, CV/ITS still cannot be massively\ndeployed in US markets due to lack of regulations, 
dedicated wireless spectrum\nbands, sustainable financial & business models with mature supply chain, etc.\nIn the 
other side, technology-driven AV market has been much slower than\nexpected mainly because of immaturity of AI 
technology to handle different\ncomplex driving scenarios in a cost effective way. In this paper, we first\npresent
these two parallel journeys focusing on the deployments including\noperating models, scenarios and applications, 
evaluations and lessons learning.\nThen, come up with recommendations to a cooperative CAV approach driving a 
more\nfeasible, safer, affordable and cost effective transportation, but require a\ngreat industry collaboration 
from Automotive, Transportation. ICT and Cloud.', 'id': 'http://arxiv.org/abs/2008.04379v1', 'categories': 
['cs.CY', 'cs.LG']}, {'title': 'On Designing Computing Systems for Autonomous Vehicles: a PerceptIn Case Study', 
'authors': ['Bo Yu', 'Jie Tang', 'Shaoshan Liu'], 'year': 2020, 'abstract': "PerceptIn develops and commercializes 
autonomous vehicles for micromobility\naround the globe. This paper makes a holistic summary of 
PerceptIn's\ndevelopment and operating experiences. This paper provides the business tale\nbehind our product, and 
presents the development of the computing system for\nour vehicles. We illustrate the design decision made for the 
computing system,\nand show the advantage of offloading localization workloads onto an FPGA\nplatform.", 'id': 
'http://arxiv.org/abs/2011.06277v1', 'categories': ['cs.RO']}, {'title': 'Towards Fully Intelligent Transportation 
through Infrastructure-Vehicle Cooperative Autonomous Driving: Challenges and Opportunities', 'authors': ['Shaoshan
Liu', 'Bo Yu', 'Jie Tang', 'Qi Zhu'], 'year': 2021, 'abstract': 'The infrastructure-vehicle cooperative autonomous 
driving approach depends on\nthe cooperation between intelligent roads and intelligent vehicles. This\napproach is 
not only safer but also more economical compared to the traditional\non-vehicle-only autonomous driving approach. 
In this paper, we introduce our\nreal-world deployment experiences of cooperative autonomous driving, and 
delve\ninto the details of new challenges and opportunities. Specifically, based on\nour progress towards 
commercial deployment, we follow a three-stage development\nroadmap of the cooperative autonomous driving 
approach:infrastructure-augmented\nautonomous driving (IAAD), infrastructure-guided autonomous driving (IGAD), 
and\ninfrastructure-planned autonomous driving (IPAD).', 'id': 'http://arxiv.org/abs/2103.02176v1', 'categories': 
['cs.RO', 'cs.AR', 'cs.NI']}]

Out: None

[Step 0: Duration 3.72 seconds| Input tokens: 2,416 | Output tokens: 116]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing this code: ────────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(report)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: # Business Opportunity Analysis Report

## Executive Summary
Analysis based on 3 research publications from 2020 to 2021, focusing on market potential and business 
opportunities.

## Market Overview
### Recent Developments
- (2021) Towards Fully Intelligent Transportation through Infrastructure-Vehicle Cooperative Autonomous Driving: 
Challenges and Opportunities
- (2020) A Survey and Insights on Deployments of the Connected and Autonomous Vehicles in US
- (2020) On Designing Computing Systems for Autonomous Vehicles: a PerceptIn Case Study

## Market Potential
### Target Market Segments
- Cloud and ict providers
- Automotive manufacturers
- Transportation and logistics companies

### Potential Business Models
- Data services through improved transportation infrastructure
- Hardware sales of autonomous vehicle components
- Subscription-based services for autonomous driving capabilities

### Market Drivers
- Safety improvements through autonomous vehicles
- Enhanced traffic efficiency
- Environmental benefits

## Technology Readiness Assessment

### Market-Ready Solutions
- DHL, FedEx, UPS

### In Development (Prototype Stage)
- Microsoft Azure, Amazon Web Services, Google Cloud

### Early Research Stage
- Tesla, Waymo, General Motors

## Risks and Challenges
- Lack of regulations
- Immaturity of ai technology
- Dedicated wireless spectrum bands needed

## Recommendations
Based on the analysis:
- Technology still in early stages
- Consider R&D investments and strategic partnerships

[Step 4: Duration 3.08 seconds| Input tokens: 23,299 | Output tokens: 2,962]

# Business Opportunity Analysis Report

## Executive Summary
Analysis based on 3 research publications from 2020 to 2021, focusing on market potential and business opportunities.

## Market Overview
### Recent Developments
- (2021) Towards Fully Intelligent Transportation through Infrastructure-Vehicle Cooperative Autonomous Driving: Challenges and Opportunities
- (2020) A Survey and Insights on Deployments of the Connected and Autonomous Vehicles in US
- (2020) On Designing Computing Systems for Autonomous Vehicles: a PerceptIn Case Study

## Market Potential
### Target Market Segments
- Cloud and ict providers
- Automotive manufacturers
- Transportation and logistics companies

### Potential Business Models
- Data services through improved transportation infrastructure
- Hardware sales of autonomous vehicle components
- Subscription-based services for autonomous driving capabilities

### Market Drivers
- Safety improvements through autonomous vehicles
- Enhanced traffic efficiency
- 